In [1]:
import pandas as pd
import numpy as np
from wutils.general import load_pickle, save_pickle
import swifter
import seaborn as sns
import matplotlib.pyplot as plt
import ujson
from collections import Counter

In [2]:
with open('./data/all_claims.json', 'r') as f:
    claims = ujson.load(f)

In [3]:
with open('./data/all_claims.json', 'r') as f:
    claims_df = pd.read_json(f)

In [4]:
GOOD_WORDS = ['true', 'correct', 'accurate ']
def classify_claim(claim):
    l_claim = claim.lower()
    if 'false' in l_claim or 'pants on fire' in l_claim or 'fake' in l_claim or 'inaccurate' in l_claim or 'incorrect' in l_claim or 'unsubstantiated' in l_claim:
        return 'false'
    if 'misleading' in l_claim:
        return 'misleading'
    if 'true' in l_claim or 'correct' in l_claim or 'accurate' in l_claim:
        return 'true'
    print('Unknown:', claim)
    return 'unknown'

def mean_claims(claims):
    c = Counter(classify_claim(x['textualRating']) for x in claims)
    return c.most_common(1)[0][0]

In [5]:
claims[0]

{'text': 'WHO DG Tedros Ghebreyesus tests positive for Covid-19',
 'claimant': 'The Nation',
 'claimDate': '2020-11-02T00:00:00Z',
 'claimReview': [{'publisher': {'name': 'AFP Fact Check',
    'site': 'factcheck.afp.com'},
   'url': 'https://factcheck.afp.com/who-head-did-not-say-he-tested-positive-covid-19-only-he-was-quarantining-after-contact-tested',
   'title': 'WHO head did not say he tested positive for Covid-19, only that he was quarantining after a contact tested positive',
   'reviewDate': '2020-11-02T17:52:00Z',
   'textualRating': 'False',
   'languageCode': 'en'}]}

In [6]:
claims_df['label'] = claims_df['claimReview'].swifter.apply(mean_claims)

Unknown: Missing Context
Unknown: No world doctors are not of WHO
Unknown: The video is not from 1956, it’s from February 2020.
Unknown: C
Unknown: Lacks_Context
Unknown: Lacks_Context
Unknown: He did invent PCR, which is a process used to test whether someone currently has Covid-19. He did die in August 2019.
Unknown: This is a flawed comparison, as the number of deaths from Covid-19 far exceeds the number of deaths from flu in a given season, despite the much stricter controls in place.
Unknown: Kary Mullis died on 7 August 2019, months before the first case of COVID-19 was reported to the World Health Organization in December of that year.
Unknown: Commotion
Unknown: Satire
Unknown: Flawed_Reasoning
Unknown: Flawed_Reasoning
Unknown: Cherry-picked number
Unknown: These drugs are COVID-19 treatments, not vaccines, and they are available in Western countries
Unknown: No evidence
Unknown: Suspicious
Unknown: This comes from the faster end of a range of estimates.
Unknown: There is no d

In [7]:
for row in claims_df.itertuples():
    print(row.label, [x['textualRating'] for x in row.claimReview])

false ['False']
false ['False']
false ['False']
false ['False']
unknown ['Missing Context']
false ['This is incorrect. At the time the claim was made, the number of deaths reported of people who had tested positive for Covid-19 and died within 28 days was 169.']
false ['Inaccurate (no factual basis; unacceptable margin of error)']
false ['Inaccurate (no factual basis; unacceptable margin of error)']
unknown ['No world doctors are not of WHO']
true ['Half true']
false ['Incorrect. The US CDC published figures suggesting the chance of dying from Covid-19 after being infected was 0.26% and has since updated these figures. The chance of catching Covid-19 is much higher than that in many countries, including the UK.']
false ['False']
false ['False']
unknown ['The video is not from 1956, it’s from February 2020.']
false ['False']
true ['Half true']
false ['This is incorrect, based on the data Mr Hancock mentioned, which suggests that people who suffer from “long Covid” had more symptoms when

In [8]:
output = []
claims[0]

{'text': 'WHO DG Tedros Ghebreyesus tests positive for Covid-19',
 'claimant': 'The Nation',
 'claimDate': '2020-11-02T00:00:00Z',
 'claimReview': [{'publisher': {'name': 'AFP Fact Check',
    'site': 'factcheck.afp.com'},
   'url': 'https://factcheck.afp.com/who-head-did-not-say-he-tested-positive-covid-19-only-he-was-quarantining-after-contact-tested',
   'title': 'WHO head did not say he tested positive for Covid-19, only that he was quarantining after a contact tested positive',
   'reviewDate': '2020-11-02T17:52:00Z',
   'textualRating': 'False',
   'languageCode': 'en'}]}

In [9]:
for claim in claims:
    for review in claim['claimReview']:
#         print(claim)
        output.append({
            'text': claim['text'],
            'claimant': claim['claimant'] if 'claimaint' in claim else None,
            'claimDate': claim['claimDate'] if 'claimDate' in claim else None,
            'claimReview': review
        })

In [10]:
len(output)

932

In [88]:
save_pickle(claims_df, 'pickles/newest_claims_df_labelled.pkl')

In [11]:
claims_df = load_pickle('pickles/newest_claims_df_labelled.pkl')

In [12]:
# cord_df = load_pickle('./pickles/newest_cord_df_ft.pkl').dropna()

In [13]:
cord_df = load_pickle('./pickles/who_cord_df.pkl').dropna()

In [14]:
# cord_df['mean_embed'] = cord_df['ft_embeddings'].swifter.apply(lambda x: np.array(x).mean(axis=0))

In [31]:
cord_df['mean_embed'] = cord_df['sent_embeddings'].swifter.apply(lambda x: x.mean(axis=0))

In [32]:
mat_cord = np.array(list(cord_df.sample(n=10000)['mean_embed']))

In [33]:
false_df = claims_df[claims_df['label'] == 'false']
true_df = claims_df[claims_df['label'] == 'true']

In [34]:
true_mat = np.array(list(true_df['ft_embedding']))

In [35]:
false_mat = np.array(list(false_df['ft_embedding']))

In [36]:
true_mat = np.array(list(true_df['bert_embedding']))
false_mat = np.array(list(false_df['bert_embedding']))

In [37]:
false_mat.shape

(562, 768)

In [38]:
true_mat.shape

(97, 768)

In [39]:
from wutils.mat import MarkedMatrix
mm = MarkedMatrix((('true', true_mat), ('false', false_mat)))

In [40]:
from sklearn.linear_model import LogisticRegression

In [41]:
from scipy.spatial.distance import cdist

In [42]:
true2cord = cdist(true_mat, mat_cord, 'cosine')
false2cord = cdist(false_mat, mat_cord, 'cosine')

In [43]:
mmb= MarkedMatrix((('true', true2cord), ('false', false2cord)))

In [84]:
mm2 = MarkedMatrix((('true', true2cord), ('false', false2cord)))

In [89]:
mm3 = MarkedMatrix((('true', true2cord), ('false', false2cord)))

In [104]:
mm4 = MarkedMatrix((('true', true_mat), ('false', false_mat)))

In [44]:
from tqdm.notebook import tqdm

In [45]:
model = LogisticRegression(max_iter=3000)
res1 = []
for i in tqdm(range(2000)):
    res1.append(mm.single_split_classify(model, verbose=False))

In [76]:
model2 = LogisticRegression(max_iter=3000)
res2 = []
for i in tqdm(range(2000)):
    res2.append(mm2.single_split_classify(model2, verbose=False))

In [90]:
model3 = LogisticRegression(max_iter=3000)
res3 = []
for i in tqdm(range(2000)):
    res3.append(mm3.single_split_classify(model, verbose=False))

In [105]:
model4 = LogisticRegression(max_iter=3000)
res4 = []
for i in tqdm(range(1000)):
    res4.append(mm4.single_split_classify(model, verbose=False))

In [47]:
np.array(res1).mean(axis=0)

array([0.79884545, 0.88497882])

In [78]:
np.array(res2).mean(axis=0)

array([0.24677551, 0.23806656])

In [46]:
np.array(res3).mean(axis=0)

NameError: name 'res3' is not defined

In [106]:
np.array(res4).mean(axis=0)

array([0.80115758, 0.88637263])

In [ ]:
labels = claims_df['label'].swifter.apply(lambda x: )